In [2]:
from typing import *
import collections
import math
import heapq
import functools

# 6188. Sort the People

In [19]:
class Solution:
    def sortPeople(self, names: List[str], heights: List[int]) -> List[str]:
        maps = {}
        for i in range(len(names)):
            maps[heights[i]] = names[i]
        ret = []
        heights.sort(reverse=True)
        for n in heights:
            ret.append(maps[n])
        return ret

In [20]:
names = ["Alice","Bob","Bob"]
heights = [155,185,150]

In [21]:
a = Solution().sortPeople(names, heights)
a

['Bob', 'Alice', 'Bob']

# 6189. Longest Subarray With Maximum Bitwise AND

In [35]:
class Solution:
    def longestSubarray(self, nums: List[int]) -> int:
        target = max(nums)
        ret = 1
        sums = nums[0]
        s = 1
        for i in range(1, len(nums)):
            if sums & nums[i] == target:
                s += 1
            else:
                ret = max(ret, s)
                sums = nums[i]
                s = 1   
        ret = max(ret, s)
        return ret

In [36]:
nums = [1,2,3,3,2,2]

In [37]:
a = Solution().longestSubarray(nums)
a

2

# 6190. Find All Good Indices

In [63]:
class Solution:
    def goodIndices(self, nums: List[int], k: int) -> List[int]:
        l = [0] * len(nums)
        r = [0] * len(nums)
        l[0] = 1
        r[-1] = 1
        for i in range(1, len(nums)):
            if nums[i] <= nums[i-1]:
                l[i] = l[i-1] + 1
            else:
                l[i] = 1
        for i in range(len(nums)-2, -1, -1):
            if nums[i] <= nums[i+1]:
                r[i] = r[i+1] + 1
            else:
                r[i] = 1
        
        ret = []
        for i in range(k, len(nums)-k):
            if l[i-1] >= k and r[i+1] >= k:
                ret.append(i)
        return ret

In [64]:
nums = [440043,276285,336957]
k = 1

In [65]:
a = Solution().goodIndices(nums, k)
a

[1]

# 6191. Number of Good Paths

In [137]:
class Solution:
    def numberOfGoodPaths(self, vals: List[int], edges: List[List[int]]) -> int:
        if len(vals) == 1:
            return 1
        maps = collections.defaultdict(list)
        for edge in edges:
            maps[edge[0]].append(edge[1])
            maps[edge[1]].append(edge[0])
        
        for key in maps.keys():
            if len(maps[key]) == 1:
                start = key
                break
        
        visited = set()
        ret = 0
        visited.add(start)
        def xunzhao(idx):
            m = []
            nonlocal ret
            for nxt in maps[idx]:
                if nxt in visited:
                    continue
                visited.add(nxt)
                l = xunzhao(nxt)
                m.append(l)
            
            newmap = collections.defaultdict(int)
            for l in m:
                for key, val in l.items():
                    if key > vals[idx]:
                        if newmap.get(key, -1) != -1:
                            ret += newmap[key] * val
                        newmap[key] += val
                    elif key == vals[idx]:
                        if newmap.get(key, -1) != -1:
                            ret += newmap[key] * val
                        newmap[key] += val 
                        ret += val
                        
            newmap[vals[idx]] += 1
            return newmap
        
        xunzhao(start)
        return ret + len(vals)

In [129]:
vals = [1,3,2,1,3]
edges = [[0,1],[0,2],[2,3],[2,4]]

In [168]:
class Union:
    def __init__(self, lens):
        self.Union = list(range(lens))
        self.rank = [1] * lens
        
    def find(self, x):
        while self.Union[x] != x:
            x = self.Union[x]
        return x

    def union(self, par_x, par_y):
        self.Union[par_y] = par_x

In [178]:
class Solution:
    def numberOfGoodPaths(self, vals: List[int], edges: List[List[int]]) -> int:
        union = Union(len(vals))
        maps = collections.defaultdict(list)
        for edge in edges:
            maps[edge[0]].append(edge[1])
            maps[edge[1]].append(edge[0])
        
        ret = 0
        for v, idx in sorted(zip(vals, range(len(vals)))):
            parv = union.find(idx)
            for nxt in maps[idx]:
                parn = union.find(nxt)
                if parv == parn or v < vals[nxt]:
                    continue
                if vals[parv] == vals[parn]:
                    ret += union.rank[parv] * union.rank[parn]
                    union.rank[parv] += union.rank[parn]
                union.union(parv, parn)
        return ret + len(vals)

In [183]:
a = Solution().numberOfGoodPaths(vals, edges)
a

52

In [180]:
vals = [1,4,5,2,4,4,1]
edges = [[0,1],[1,2],[1,3],[4,2],[5,1],[5,6]]

In [181]:
vals = [3,2,1,2,5,2,4,5,1,5]
edges = [[0,1],[1,2],[1,3],[4,3],[4,5],[6,0],[0,7],[8,1],[2,9]]

In [182]:
vals = [16,16,16,6,12,9,6,2,17,7,5,15,6,3,14,6,16,3,16,11,9,4,15,16,2,12,7,7,9,7,3,8,4,13,8,4]
edges = [[1,0],[2,0],[3,1],[0,4],[5,1],[4,6],[2,7],[8,3],[9,1],[10,1],[11,2],[6,12],[13,0],[7,14],[15,11],[12,16],[17,1],[11,18],[19,11],[20,12],[21,0],[22,18],[23,2],[24,2],[25,21],[10,26],[27,25],[17,28],[19,29],[5,30],[31,8],[32,13],[20,33],[21,34],[35,15]]

In [175]:
class Solution:
    def numberOfGoodPaths(self, vals: List[int], edges: List[List[int]]) -> int:
        n = len(vals)
        g = [[] for _ in range(n)]
        for x, y in edges:
            g[x].append(y)
            g[y].append(x)  # 建图

        # 并查集模板
        fa = list(range(n))
        # size[x] 表示节点值等于 vals[x] 的节点个数，如果按照节点值从小到大合并，size[x] 也是连通块内的等于最大节点值的节点个数
        size = [1] * n
        def find(x: int) -> int:
            if fa[x] != x:
                fa[x] = find(fa[x])
            return fa[x]

        ans = n
        for vx, x in sorted(zip(vals, range(n))):
            fx = find(x)
            for y in g[x]:
                y = find(y)
                if y == fx or vals[y] > vx: continue  # 只考虑最大节点值比 vx 小的连通块
                if vals[y] == vx:  # 可以构成好路径
                    ans += size[fx] * size[y]  # 乘法原理
                    size[fx] += size[y]  # 统计连通块内节点值等于 vx 的节点个数
                fa[y] = fx  # 把小的节点值合并到大的节点值上
        return ans

In [3]:
# 让高度小的树成为高度较大的树的子树，这个优化可以称为启发式合并算法。并查集的按秩合并就是启发式合并算法的一种
class Solution:
    def numberOfGoodPaths(self, vals: List[int], edges: List[List[int]]) -> int:
        def dfs(cur: int, pre: int) -> "Dict[int, int]":
            """后序dfs返回子树内的每种结点个数"""
            self.res += 1
            curRes = {vals[cur]: 1}

            for next in adjList[cur]:
                if next == pre:
                    continue
                nextRes = dfs(next, cur)
                for key in list(nextRes):
                    if key < vals[cur]:
                        del nextRes[key]

                if len(curRes) < len(nextRes):  # !启发式合并
                    curRes, nextRes = nextRes, curRes
                for key in nextRes:
                    self.res += curRes.get(key, 0) * nextRes[key]

                curRes.update({key: curRes.get(key, 0) + nextRes[key] for key in nextRes})

            return curRes

        n = len(vals)
        adjList = [[] for _ in range(n)]
        for u, v in edges:
            adjList[u].append(v)
            adjList[v].append(u)

        self.res = 0
        dfs(0, -1)
        return self.res